# Model Inspection

In [6]:
from learning.models import *
from torchinfo import summary

Define a model

In [7]:
model = FCN(num_channels=9,
            window_size=120,
            future_size=20,
            hidden_sizes=None,
            channel_sizes=[128, 128, 128],
            kernel_sizes=[3, 5, 7, 5, 3],
            stride_sizes=[1, 1, 1, 1, 1],
            dilation_sizes=[1, 1, 1],
            pool_sizes=[2, 2, 2, 1, 1],
            state_sizes=[64, 64, 64],
            attn_sizes=[128, 128, 128],
            head_sizes=[4, 4, 4],
            dropout_rate=0,
            batch_normalization=False)

Print the model's components

In [8]:
print(model)

FCN(
  (conv_stack): ConvStack(
    (conv0): Conv1d(9, 128, kernel_size=(3,), stride=(1,), padding=(1,))
    (relu0): ReLU()
    (max_pool0): MaxPool1d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (conv1): Conv1d(128, 128, kernel_size=(5,), stride=(1,), padding=(2,))
    (relu1): ReLU()
    (max_pool1): MaxPool1d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (conv2): Conv1d(128, 128, kernel_size=(7,), stride=(1,), padding=(3,))
    (relu2): ReLU()
    (max_pool2): MaxPool1d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (out_conv): Conv1d(128, 700, kernel_size=(1,), stride=(1,))
  (gap): AvgPool1d(kernel_size=(15,), stride=(15,), padding=(0,))
  (unflatten): Unflatten(dim=-1, unflattened_size=(5, 140))
)


Analyze the parameter sizes

In [9]:
summary(model, input_size=(1, 9, 120), device="cpu", depth=2)

Layer (type:depth-idx)                   Output Shape              Param #
FCN                                      --                        --
├─ConvStack: 1-1                         [1, 128, 15]              --
│    └─Conv1d: 2-1                       [1, 128, 120]             3,584
│    └─ReLU: 2-2                         [1, 128, 120]             --
│    └─MaxPool1d: 2-3                    [1, 128, 60]              --
│    └─Conv1d: 2-4                       [1, 128, 60]              82,048
│    └─ReLU: 2-5                         [1, 128, 60]              --
│    └─MaxPool1d: 2-6                    [1, 128, 30]              --
│    └─Conv1d: 2-7                       [1, 128, 30]              114,816
│    └─ReLU: 2-8                         [1, 128, 30]              --
│    └─MaxPool1d: 2-9                    [1, 128, 15]              --
├─Conv1d: 1-2                            [1, 700, 15]              90,300
├─AvgPool1d: 1-3                         [1, 700, 1]               --